### row < 10,000 일 경우

In [ ]:
# 데이터 프레임 관련
import numpy  as np
import pandas as pd

# API 호출 관련
import json
import requests

# module
import psycopg2 # db connect

In [ ]:
# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [ ]:
# DB Connect
conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)

In [ ]:
# execute
execute_mogrify(conn, DF_final, table_nm, val)

### row >= 10,000 일 경우

In [ ]:
# 데이터 프레임 관련
import numpy  as np
import pandas as pd

# API 호출 관련
import json
import requests

# db connect
import psycopg2 

In [ ]:
# df 명, table 명 입력

In [ ]:
# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

### INSERT 간략

In [ ]:
# module
import psycopg2 # db connect

In [ ]:
# DB Connect
conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)

In [ ]:
# 변수 선언
start = '2022-07'
end = '2022-07'
table = 'm1.rtms_trade'
filename = 'getRTMSDataSvcNrgTrade(202207)'
dir = 'C:/oasis/'

In [52]:
# read & null 처리
df = pd.read_csv(dir + filename + '.csv', sep='|', low_memory=False)
df = df.replace({np.nan: None})

In [53]:
# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

In [54]:
# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [55]:
# execute
execute_mogrify(conn, tmp, table, val)
tmp = pd.DataFrame()

Error: syntax error at or near ")"
LINE 1: INSERT INTO m1.rtms_trade() VALUES 
                                  ^



### KT INSERT

In [ ]:
# module
import pysftp # 압축 풀기
import psycopg2 # db connect
import pandas as pd
import numpy  as np
import zipfile  # 압축 풀기 
import datetime # 시간 출력

In [37]:
# Bulk Insert
for i in pd.date_range(start, end, freq = 'MS'):

    # SFTP Connect
    cnopts = pysftp.CnOpts()
    cnopts.hostkeys = None

    with pysftp.Connection(
            '14.63.143.72',
            port=2222,
            username='oasisuser',
            password='4FTPuser1!',
            cnopts=cnopts) as sftp:

        # file 내려 받기
        sftp.get(filename + i.strftime("%Y%m") + '.zip', dir + filename + i.strftime("%Y%m") + '.zip')

    # 압축 풀기
    zipfile.ZipFile(dir + filename + i.strftime("%Y%m") + '.zip').extractall(dir)

    # read & null 처리
    df = pd.read_csv(dir + filename + i.strftime("%Y%m") + '.csv', sep='|', low_memory=False)
    df = df.replace({np.nan: None})

    # values 설정
    val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

    # execute_mogrify
    def execute_mogrify(conn, df, table, val):
        # Create a list of tuples from the dataframe values
        tuples = [tuple(x) for x in df.to_numpy()]
        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        # SQL query to execute
        cursor = conn.cursor()
        values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                  tuples]
        query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

        try:
            cursor.execute(query, tuples)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()

    # insert
    for j in range(0, len(df), 10000):
        tmp = df[j:j + 10000]
        print(datetime.datetime.now(), ' : ', j)
        execute_mogrify(conn, tmp, table, val)
        tmp = pd.DataFrame()


    # row count
    print(len(df))

FileNotFoundError: [Errno 2] No such file